In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
import json
import random
import re

In [3]:
df_orginal_android = pd.read_excel('./data/qb/original/android.xlsx')

In [4]:
len(df_orginal_android)

12281

In [ ]:
df_orginal_android.head()

In [ ]:
#Import original data
df_orginal_ios = pd.read_excel('./data/qb/original/ios.xlsx')

In [ ]:
df_orginal_ios.head()

In [ ]:
df_orginal = df_orginal_ios.append(df_orginal_android, ignore_index=True)

In [ ]:
#Get selective columns

df = df_orginal[['Review ID','Country', 'Version', 'Rating', 'Date', 'Subject', 'Body', 'Sentiment']]

In [ ]:
df.head()

In [ ]:
df['Subject'] = df['Subject'].replace(np.nan, '', regex=True)
df['Body'] = df['Body'].replace(np.nan, '', regex=True)

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
#combine subject and body
#df['Review'] = df[['Subject', 'Body']].apply(lambda x: '. '.join(x), axis=1)

for index, row in df.iterrows():
    sub = row['Subject']
    body = row['Body']
    
    df.loc[index,'Review'] = str(sub) + ". " + str(body)

In [ ]:
df.head()

In [ ]:
#Drop Subject and Body

del df['Subject']
del df['Body']

In [ ]:
df.head()

In [21]:
filename = "./data/qb/processed/train_ner.json"
print(filename)


with open(filename) as train_data:
    train = json.load(train_data)

TRAIN_DATA_NEW = []
for data in train:
    ents = [tuple(entity) for entity in data['entities']]
    TRAIN_DATA_NEW.append((data['content'],{'entities':ents}))


with open('{}'.format(filename.replace('json','txt')),'w') as write:
    write.write(str(TRAIN_DATA_NEW))


./data/qb/processed/train_ner.json


In [22]:

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.load('en_core_web_sm')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA_NEW, 20)

Statring iteration 0
{'ner': 2026.224508466591}
Statring iteration 1
{'ner': 2028.0348857634299}
Statring iteration 2
{'ner': 1888.9462946534277}
Statring iteration 3
{'ner': 2158.9921128757787}
Statring iteration 4
{'ner': 1980.4446047670474}
Statring iteration 5
{'ner': 2100.2030422435587}
Statring iteration 6
{'ner': 1918.9685051096747}
Statring iteration 7
{'ner': 1965.7143093003533}
Statring iteration 8
{'ner': 1842.3065502064946}
Statring iteration 9
{'ner': 1920.9444526238128}
Statring iteration 10
{'ner': 1868.9978669467166}
Statring iteration 11
{'ner': 1948.4514279842679}
Statring iteration 12
{'ner': 1936.9918960441307}
Statring iteration 13
{'ner': 1906.0687236476688}
Statring iteration 14
{'ner': 1847.6175456084263}
Statring iteration 15
{'ner': 1940.7462582741741}
Statring iteration 16
{'ner': 1810.2830493400627}
Statring iteration 17
{'ner': 1905.0144225298377}
Statring iteration 18
{'ner': 1927.987444192847}
Statring iteration 19
{'ner': 1784.5002934570007}


In [23]:
# Save our trained Model
prdnlp.to_disk("./model/qbo_aspect")

In [24]:
#Test your text
test_text = ["Cancelling your subscription seems impossible. I've emailed, called, used their help menu. It shouldn't be this hard. Still trying to cancel.",
              "My App has stopped working on my Samsung s9 for 2 days in a row. You should have sent an email to inform subscribers if your app is going to be experiencing a downtime.",
              "Cannot open customer invoices. Please correct ASAP!",
               "bugs on iOS13 please update ASAP!"]
for text in test_text:
    print('-----')
    doc = prdnlp(text)
    for ent in doc.ents:
        print('keyword=' + ent.text, 'aspect=' + ent.label_)

-----
keyword=subscription aspect=Subscription
keyword=hard aspect=Experience
-----
keyword=App aspect=App
-----
keyword=invoices aspect=Invoicing
-----
keyword=bugs aspect=Experience
keyword=update aspect=Updates


In [25]:
for index, row in df.iterrows():
    aspect_terms = []
    aspect_related_text = []
    doc = prdnlp(row['Review'])
    for ent in doc.ents:
        if ent.label_ not in aspect_terms:            
            aspect_terms.append(ent.label_)
            aspect_related_text.append(str(ent))

    df.loc[index,'Aspects'] = ", ".join(aspect_terms)
    df.loc[index,'AspectText'] = ", ".join(aspect_related_text)

In [18]:
df.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,AspectText
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...,"App, Expenses, Help, Customer Service","app, expenses, videos, support"
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...,App,app
2,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...,"App, Transactions, Subscription","App, transaction, subscription"
3,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...,App,apps
4,5.467691e+09,USA,20.01.6,5,2020-01-30,positive,Small business. This absolutely works perfectl...,,


In [26]:
#number of reviews without aspect
(df['Aspects'].values == '').sum() 

7193

In [27]:
len(df)

19841

In [28]:
df['Aspects'].replace('', np.nan, inplace=True)
df.dropna(subset=['Aspects'], inplace=True)

/home/rmajumder/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/rmajumder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
len(df)

12648

In [30]:
df.head(50)

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,AspectText
0,5.46799e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...,"App, Expenses","useless, expenses"
1,5.46424e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...,"App, Updates","app, update"
2,5.4671e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...,"App, Expenses, Experience, Subscription","App, expenses, department, subscription"
3,5.46699e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...,App,apps
5,5.46055e+09,USA,20.01.4,1,2020-01-29,negative,Worst update yet. Just deleted. Crashed. Didn’...,"Updates, Experience","update, deleted"
6,5.46148e+09,USA,20.01.4,1,2020-01-29,negative,Transferring to TurboTax. Transferring company...,Experience,issue
7,5.46253e+09,USA,20.01.4,5,2020-01-29,positive,Great app for small business owners!. I have a...,App,app
8,5.46281e+09,USA,20.01.4,4,2020-01-29,mixed,Good but needs multi currency. Very easy to us...,Invoicing,invoice
10,5.45735e+09,United Kingdom,20.01.4,4,2020-01-28,neutral,Good App but. It’s a good app but you should b...,"App, Invoicing","App, invoices"
11,5.45882e+09,USA,20.01.4,3,2020-01-28,negative,Having issues with iPhone 11. After installing...,App,app


In [31]:
!pip install afinn

You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:
from afinn import Afinn
af = Afinn()

In [33]:
#generate aspect position index

def generate_df_with_position(data_frame):

    df_split_terms = pd.DataFrame(columns=['Review ID', 
                                           'Country', 'Version', 
                                           'Rating', 'Date', 'Doc Sentiment', 'Asp Sentiment', 
                                           'Review', 'Aspects', 'AspectText', 'Positions'])


    for index, row in data_frame.iterrows():
        aspecttexts = [x.strip() for x in row['AspectText'].lower().split(',')]
        aspects = [x.strip() for x in row['Aspects'].lower().split(',')]

        positions = ''
        for idx, asp in enumerate(aspecttexts):

            review = " ".join(row['Review'].lower().split())

            if len(review.split(' ')) > 85:
                continue

            start_idx = review.find(asp)

            #if the aspect is not found in the review, skip
            if start_idx == -1:
                continue

            sentiment = 'neutral' if row['Sentiment'] == 'mixed' else row['Sentiment']
            
            
            df_split_terms = df_split_terms.append({
                'Review ID': row['Review ID'], 
                'Country': row['Country'], 
                'Version': row['Version'],
                'Rating': row['Rating'],
                'Date': row['Date'],            
                'Doc Sentiment': sentiment,
                'Asp Sentiment' : '' if len(aspecttexts) > 1 else sentiment,
                'Review': review,
                'AspectText': asp,
                'Aspects': aspects[idx] if len(aspects) > idx else asp,
                'Positions': str(start_idx) + ',' + str(start_idx + len(asp) - 1),
            }, ignore_index=True)

    return df_split_terms

In [34]:
df_split_terms = generate_df_with_position(df)
df_split_terms.head()

,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,5.46424e+09,Canada,20.01.5,1,2020-01-30,negative,,latest app update. this update is terrible bas...,app,app,"7,9"
1,5.46424e+09,Canada,20.01.5,1,2020-01-30,negative,,latest app update. this update is terrible bas...,updates,update,"11,16"
2,5.46699e+09,USA,20.01.5,5,2020-01-30,positive,positive,life saver. one my favorite apps to use while ...,app,apps,"28,31"
3,5.46055e+09,USA,20.01.4,1,2020-01-29,negative,,worst update yet. just deleted. crashed. didn’...,updates,update,"6,11"
4,5.46055e+09,USA,20.01.4,1,2020-01-29,negative,,worst update yet. just deleted. crashed. didn’...,experience,deleted,"23,29"


In [35]:
len(df_split_terms)

17357

In [36]:

df_split_terms.to_csv(r'data/qb/processed/combine_aspect_position.csv')

In [22]:
for index, row in df_position_sentiment.iterrows():
    if (row['Rating'] == 1 or row['Rating'] == 5) and row['Asp Sentiment'] == 0:
        df_position_sentiment.loc[index,'Asp Sentiment'] = row['Doc Sentiment']
    

In [24]:
#number of reviews without aspect
(df_position_sentiment['Asp Sentiment'].values == 0).sum() 

2918

In [2]:

#df_position_sentiment = pd.read_csv(r'./data/qb/processed/combine_aspect_position_14000.csv')

In [21]:
df_position_sentiment = pd.read_csv(r'./data/qb/processed/combine_aspect_position_0325_clean.csv')

In [2]:
df_position_sentiment = pd.read_csv(r'./data/qb/processed/combine_aspect_position_0408_nolemma_clean.csv')

In [3]:
df_position_sentiment['Asp Sentiment'].replace('0', np.nan, inplace=True)

In [4]:
df_position_sentiment.dropna(subset=['Asp Sentiment'], inplace=True)

In [6]:
#df_position_sentiment = df_position_sentiment.head(3000)
#(df_position_sentiment['Doc Sentiment'].values != df_position_sentiment['Asp Sentiment'].values).sum() 

In [7]:
#df_position_sentiment.drop(
#    df_position_sentiment[df_position_sentiment['Doc Sentiment'].values == df_position_sentiment['Asp Sentiment']
#                          .values].index, inplace=True)

In [24]:
df_position_sentiment.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,0,0,0,5464235433,Canada,20.01.5,1,2020-01-30,negative,neutral,latest app update. this update is terrible bas...,app,app,"7,9"
1,1,1,1,5464235433,Canada,20.01.5,1,2020-01-30,negative,negative,latest app update. this update is terrible bas...,updates,update,"11,16"
2,2,2,2,5466985498,USA,20.01.5,5,2020-01-30,positive,positive,life saver. one my favorite apps to use while ...,app,apps,"28,31"
3,3,3,3,5460552711,USA,20.01.4,1,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,updates,update,"6,11"
4,4,4,4,5460552711,USA,20.01.4,1,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,experience,deleted,"23,29"


In [5]:
#df_position_sentiment[df_position_sentiment.AspectText == ]
print(len(df_position_sentiment))
df_position_sentiment.dropna(subset=['AspectText'], inplace=True)
print(len(df_position_sentiment))

14824
14821


In [6]:
df_position_sentiment['Asp Sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [7]:
#recalculate position
for index, row in df_position_sentiment.iterrows():
    
    df_position_sentiment.loc[index,'Review'] = row['Review'].replace('/', '').replace('-', '')
    
    start_idx = row['Review'].find(row['AspectText'])
          
    df_position_sentiment.loc[index,'Positions'] = str(start_idx) + ',' + str(start_idx + len(row['AspectText']) - 1)
    

In [8]:
df_position_sentiment.to_csv(r'data/qb/processed//combine_aspect_position_0408_clean_after.csv')

In [9]:
#df_position_sentiment[df_position_sentiment.AspectText == ]
print(len(df_position_sentiment))
df_position_sentiment.dropna(subset=['Positions'], inplace=True)
print(len(df_position_sentiment))

14821
14821


In [10]:
(df_position_sentiment['Review'].str.len() <= 500).sum()

14774

In [10]:
df_position_sentiment.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,0,0,0,5464235433,Canada,20.01.5,1,2020-01-30,negative,neutral,latest app update. this update is terrible bas...,app,app,"7,9"
1,1,1,1,5464235433,Canada,20.01.5,1,2020-01-30,negative,negative,latest app update. this update is terrible bas...,updates,update,"11,16"
2,2,2,2,5466985498,USA,20.01.5,5,2020-01-30,positive,positive,life saver. one my favorite apps to use while ...,app,apps,"28,31"
3,3,3,3,5460552711,USA,20.01.4,1,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,updates,update,"6,11"
4,4,4,4,5460552711,USA,20.01.4,1,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,experience,deleted,"23,29"


In [11]:
#split data in - train test and dev
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    
    perm = np.random.permutation(np.arange(len(df)))
    m = len(df)
    
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end - 1] - 1]
    validate = df.iloc[perm[train_end:validate_end - 1] - 1]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [12]:
train, validate, test = train_validate_test_split(df_position_sentiment)

In [17]:
train.head()

,Unnamed: 0,Unnamed: 0.1,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
8742,8744.0,8771.0,gp:AOqpTOGCArglMaLm9qWy06RodxO2yaV0-Xxdc8FF3kf...,English,19.11.0.8,1.0,2019-12-23,negative,negative,. why can't i see bank account details in the ...,experience,useless,"163,169"
17029,17233.0,17260.0,gp:AOqpTOErhI6pc7RWzifbysMVXjGOrE9ncGtaUTJN5w9...,English,3.3,5.0,2013-10-09,positive,positive,. full featured app! quickbooks makes it easy ...,invoicing,invoices,"128,135"
16246,16416.0,16443.0,gp:AOqpTOE7QyAWqmIhqnST3hIXOEH7LXXtIxj7_dTbG-m...,English,3.6.1,5.0,2014-08-14,positive,positive,great app. no longer have to worry about compu...,app,app,"6,8"
3517,3519.0,3546.0,1574919983,USA,5.2.2,4.0,2017-03-28,positive,positive,great app. very easy to use. reliable. every o...,app,app,"6,8"
16338,16510.0,16537.0,gp:AOqpTOFsv2ISx1GCivE4Xk8J4V6FYUjFmFLbMBGJdrn...,English,3.6,4.0,2014-07-12,positive,positive,pretty convenient!. i quite like being able to...,app,app,"83,85"


In [13]:
print(len(train))
print(len(test))
print(len(validate))

8891
2965
2963


### Generate files

In [14]:
def generate_input_files(d_type, data):

    #save reviews
    df_review = data[['Review']]
    df_review.to_csv(r'data/qb/{}/review.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save aspects
    df_review = data[['AspectText']]
    df_review.to_csv(r'data/qb/{}/term.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save positions
    df_review = data[['Positions']]
    df_review.to_csv(r'data/qb/{}/position.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save label
    df_review = data[['Asp Sentiment']]
    df_review.to_csv(r'data/qb/{}/label.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

In [15]:
generate_input_files('train', train)
generate_input_files('test', test)
generate_input_files('dev', validate)